# MMAI 2025 869: Team Project Template
*Updated May 3, 2024*

This notebook serves as a template for the Team Project. Teams can use this notebook as a starting point, and update it successively with new ideas and techniques to improve their model results.

Note that is not required to use this template. Teams may also alter this template in any way they see fit.


# Preliminaries: Inspect and Set up environment

No action is required on your part in this section. These cells print out helpful information about the environment, just in case.

In [ ]:
import datetime
import pandas as pd
import numpy as np

In [ ]:
print(datetime.datetime.now())

In [ ]:
!which python

In [ ]:
!python --version

In [ ]:
!echo $PYTHONPATH

In [ ]:
! pip install --user xgboost
! pip install --user fancyimpute
! pip install --user sklearn-genetic
! pip install --user ipywidgets

# 0. Data Loading and Inspection

## 0.1: Load data

The file containing the labeled training data is conveniently located on the cloud at the address below. Let's load it up and take a look.

In [ ]:
df = pd.read_csv("https://drive.google.com/uc?export=download&id=1eYCKuqJda4bpzXBVnqXylg0qQwvpUuum")
target_feature = 'h1n1_vaccine'

## 0.1 Simple Exploratory Data Analysis

In [ ]:
df.info()

In [ ]:
# Let's print some descriptive statistics for all the numeric features.

df.describe().T


In [ ]:
# What is the number of unique values in all the categorical features? And what is
# the value with the highest frequency?

df.describe(include=object).T

In [ ]:
# How much missing data is in each feature?

df.isna().sum()

### Taking a look at the class imbalance of our target

In [ ]:
import seaborn as sns
from matplotlib import pyplot as plt
from sklearn.preprocessing import LabelEncoder,OrdinalEncoder

def get_target_skew_rate(data_target):
    target_df = pd.DataFrame(data_target)
    sns.countplot(x=target_feature, data=target_df)

    no_vaccine_count = len(target_df[target_df[target_feature]==0])
    yes_vaccine_count = len(target_df[target_df[target_feature]==1])
    print(f"No vaccine: {no_vaccine_count}")
    print(f"Has vaccine: {yes_vaccine_count}")

    # save this for later...
    training_data_pos_scale_weight = (no_vaccine_count / yes_vaccine_count)
    print(f"training_data_pos_scale_weight: {training_data_pos_scale_weight}")
    return training_data_pos_scale_weight

training_data_pos_scale_weight = get_target_skew_rate(df[target_feature])

In [ ]:
# For convienience, let's save the names of all numeric features to a list,
# and the names of all categorical features to another list.

numeric_features = [
          "h1n1_concern",
          "h1n1_knowledge",
          "behavioral_antiviral_meds",
          "behavioral_avoidance",
          "behavioral_face_mask",
          "behavioral_wash_hands",
          "behavioral_large_gatherings",
          "behavioral_outside_home",
          "behavioral_touch_face",
          "doctor_recc_h1n1",
          "doctor_recc_seasonal",
          "chronic_med_condition",
          "child_under_6_months",
          "health_worker",
          "health_insurance",
          "opinion_h1n1_vacc_effective",
          "opinion_h1n1_risk",
          "opinion_h1n1_sick_from_vacc",
          "opinion_seas_vacc_effective",
          "opinion_seas_risk",
          "opinion_seas_sick_from_vacc",
          "household_adults",
          "household_children",
]

behavioural_features = list(x for x in numeric_features if 'behavioral' in x)

categorical_features = [
    "age_group",
    "education",
    "race",
    "sex",
    "income_poverty",
    "marital_status",
    "rent_or_own",
    "employment_status",
    "hhs_geo_region",
    "census_msa",
    "employment_industry",
    "employment_occupation",
]

all_features = numeric_features + categorical_features

all_cat_features = categorical_features

continuous_features = [
    'opinion_h1n1_vacc_effective',
    'opinion_h1n1_risk',
    'opinion_h1n1_sick_from_vacc',
    'opinion_seas_vacc_effective',
    'opinion_seas_risk',
    'opinion_seas_sick_from_vacc',
]

### helper method for getting the least correlated items to the target

In [ ]:
def get_low_correlations_for_target(data, show_plot = False):
    corr = data.corr()

    threshold = 0.01
    tf_corr = corr[(abs(corr[target_feature]) <= threshold)]
    display(tf_corr)

    if show_plot:
        g = sns.heatmap(corr,  vmax=.3, center=0,
                    square=True, linewidths=1, 
                    cbar_kws={"shrink": .5}, annot=True, 
                    fmt='.2f', cmap='coolwarm')
        sns.despine()
        g.figure.set_size_inches(30,25)
        plt.show()

    return list(tf_corr.index)

## How do each of the behavioural features correlate to the target?

In [ ]:
df_bv_target = pd.concat((df[behavioural_features], df[target_feature]), axis=1)
df_bv_target.hist()
get_low_correlations_for_target(df_bv_target, show_plot=True)

# separate these by skewedness in correlation with the target
# so when we FE combine these, it makes sense
behavioural_features_neg = [
          "behavioral_avoidance",
          "behavioral_wash_hands",
          "behavioral_outside_home",
          "behavioral_touch_face",
]

behavioural_features_pos = [x for x in behavioural_features if x not in behavioural_features_neg]


# 1. Train/Test Split

Now we randomly split the available data into train and test subsets.

The training data will later be used to build and assess the model on various combinations of hyperparaters.

The testing data will be used as a "final estimate" of a model's performance.

# 2. Model 1 (A simple DecisionTree model)

As a baseline, we'll do the absolute bare minimum data cleaning and then quickly build a simple Decision Tree.

In [ ]:
from sklearn.impute import SimpleImputer
from sklearn.model_selection import cross_validate
# from sklearn.model_selection import train_test_split
import xgboost as xgb


In [ ]:
# Scikit-learn needs us to put the features in one dataframe, and the label in another.
# It's tradition to name these variables X and y, but it doesn't really matter.

X = df.drop(target_feature, axis=1)
y = df[target_feature]

# X[continuous_features].hist()
# X[behavioural_features].hist()

## 1.1 Cleaning and FE

### Feature Encoding / Label Mapping

In [ ]:
label_mapping = {}

def apply_label_mapping_fxn(row):
    global label_mapping
    for feature in all_cat_features:
        feature_mapping = label_mapping[feature]
        enc_value = feature_mapping[row[feature]]
        row[feature] = enc_value
    return row

def apply_label_mapping(data):
    return data.apply(apply_label_mapping_fxn, axis=1)

def update_row_mappings(row):
    global label_mapping
    for feature in all_cat_features:
        if not feature in label_mapping:
            label_mapping[feature] = {}
        enc_feature_name = f"{feature}_enc"
        orig_value = row[feature]
        enc_value = row[enc_feature_name]
        label_mapping[feature][orig_value] = enc_value

    return row

def set_label_mapping(orig_data, encoded_data):
    global label_mapping
    map_df = pd.DataFrame()
    for feature in all_cat_features:
        map_df[feature] = orig_data[feature]
        map_df[f"{feature}_enc"] = encoded_data[feature]
    map_df.apply(update_row_mappings, axis=1)

def label_encoding(data):
    global labeled_columns

    import category_encoders as ce
    encoder = ce.JamesSteinEncoder(cols=all_cat_features)
    labeled = encoder.fit_transform(data, y)
    labeled_columns = list(encoder.get_feature_names_out())

    set_label_mapping(data, labeled)

    return labeled


### Imputation

In [ ]:
from fancyimpute import SoftImpute

def impute_data(data):
    return pd.DataFrame(SoftImpute(verbose=False).fit_transform(data), columns=data.columns)



### Scale our skewed features (using Box-Cox)

In [ ]:
from sklearn.preprocessing import PowerTransformer
display(X[continuous_features].describe().T)

def set_non_zero(row):
    for feature in continuous_features:
        if row[feature] == 0.0:
            row[feature] = 0.00000001
    return row

def set_continuous_features(data):
    # data[continuous_features] = data[continuous_features].apply(set_non_zero, axis=1)

    display(data[continuous_features].describe().T)

    scaler = PowerTransformer(method='yeo-johnson')
    scaler.fit(data[continuous_features])
    data[continuous_features] = scaler.transform(data[continuous_features])
    # data[continuous_features].hist()
    return data

### Feature Engineering

In [ ]:
def feature_engineering(data):
    data['insured_family_size'] = data['household_adults'] + data['health_insurance'] + data['household_children']
    data['at_risk_patient'] = data['doctor_recc_h1n1'] + data['doctor_recc_seasonal'] + data['chronic_med_condition']

    data['behavioral_risk_neg'] = data[behavioural_features_neg].sum(axis=1)
    data['behavioral_risk_pos'] = data[behavioural_features_pos].sum(axis=1)
    # data = data.drop(behavioural_features, axis=1)

    data['opinion_seas_risk'] = data['opinion_seas_risk'] + data['opinion_seas_sick_from_vacc']
    data['opinion_h1n1'] = data['opinion_h1n1_vacc_effective'] + data['h1n1_concern']
    data['poverty_vs_insurance'] = data['income_poverty'] + data['health_insurance']
    return data


## Correlation mapping...



In [ ]:
heatmap_data = pd.concat([label_encoding(X), y], axis=1)
low_correlation_features = get_low_correlations_for_target(heatmap_data)


## Our primary data cleaning / preprocessing pipeline method

In [ ]:
def clean_data(data, use_fe=False):
    ret = data
    
    ret = apply_label_mapping(ret)

    ret = impute_data(ret)
    ret = set_continuous_features(ret)

    if use_fe:
        ret = feature_engineering(ret)

    # ret = drop_not_used_ga_features(ret)

    # use our full training set to get the correlation data
    # ret = ret.drop(low_correlation_features, axis=1)

    return ret

cleaned_data = clean_data(X, use_fe=False)


## 1.2 Model Creation, Hyperparameter Tuning, and Validation

In [ ]:
from sklearn.model_selection import cross_validate
from sklearn.metrics import classification_report, roc_auc_score
from sklearn import metrics
import matplotlib.pyplot as plt

def display_cv(classifier, data, truth):
    classifier.fit(data, truth)
    predicted = classifier.predict(data)

    # We use cross_validate to perform K-fold cross validation for us.
    cv_results = cross_validate(classifier, data, truth, cv=5, scoring="f1_macro")
    f1_score = np.mean(cv_results['test_score'])
    display(f"results f1: {f1_score}")

    print(classification_report(truth, predicted))
    print(f"auc: {roc_auc_score(truth, predicted)}")

    confusion_matrix = metrics.confusion_matrix(truth, predicted)
    cm_display = metrics.ConfusionMatrixDisplay(confusion_matrix = confusion_matrix, display_labels = [0, 1])
    cm_display.plot()
    plt.show()    

### Testing with a DecisionTreeClassifier 

In [ ]:
from sklearn.model_selection import GridSearchCV, RandomizedSearchCV
from sklearn.tree import DecisionTreeClassifier
from sklearn_genetic import GASearchCV
from sklearn_genetic.space import Continuous, Categorical, Integer
from sklearn_genetic.callbacks import DeltaThreshold
from sklearn_genetic.callbacks import ProgressBar

pg_bar_callback = ProgressBar()
delta_callback = DeltaThreshold(threshold=0.0001, metric='fitness')

DO_DTC_TUNING = False

def run_hypertune_test_dt(data, truth):
    search_params_to_use = {
        'criterion': Categorical(choices=['gini', 'entropy', 'log_loss'], random_state=1),
        'max_depth': Integer(3, 8, 'uniform', random_state=1),
        'class_weight': Categorical(choices=['balanced', None], random_state=1),
        'min_samples_split': Continuous(0.5, 1.0, 'uniform', random_state=1),
        'min_samples_leaf': Integer(5, 15, 'uniform', random_state=1),
        # 'min_weight_fraction_leaf': Continuous(0.2, 0.5, 'uniform', random_state=1),
        # 'max_features': Continuous(0.5, 1.0, 'uniform', random_state=1),
        #'min_impurity_decrease': Continuous(0.0, 0.5, 'uniform', random_state=1),
    }
    
    test_model = DecisionTreeClassifier(random_state=1).fit(data, truth)

    grid_search = GASearchCV(test_model, param_grid=search_params_to_use, n_jobs=-1, scoring="f1_macro")
    grid_search.fit(data, truth, callbacks=[pg_bar_callback, delta_callback])
    display(grid_search.best_score_)
    best_params = grid_search.best_params_
    display(grid_search.best_params_)

    test_model = DecisionTreeClassifier(random_state=1, **best_params).fit(data, truth)
    display_cv(test_model, data, truth)
    return best_params

best_dt_params = {}
if DO_DTC_TUNING:
    best_dt_params = run_hypertune_test_dt(cleaned_data, y)
else:
    best_dt_params = {
        'class_weight': 'balanced',
        'max_depth': 6, 
        'min_samples_leaf': 9,
        'ccp_alpha': 0
    }
    

def get_dt_classifier(data, truth, parameters):
    print(f'-- DecisionTreeClassifier using: {parameters}')
    return DecisionTreeClassifier(random_state=1, **parameters).fit(data, truth)

display(best_dt_params)
test_c = get_dt_classifier(cleaned_data, y, best_dt_params)
display_cv(test_c, cleaned_data, y)


### Testing / Tuning with HistGradientBoostingClassifier 

In [ ]:
from sklearn.ensemble import HistGradientBoostingClassifier
from sklearn_genetic import GASearchCV
from sklearn_genetic.space import Continuous, Categorical, Integer
from sklearn_genetic.callbacks import DeltaThreshold

delta_callback = DeltaThreshold(threshold=0.0001, metric='fitness')

DO_HGB_TUNING = False

def run_hypertune_test_hgb(data, truth):
    search_params_to_use = {
        'learning_rate': Continuous(0.1, 1.0, 'uniform', random_state=1),
        'class_weight': Categorical(choices=['balanced', None], random_state=1),
        'max_leaf_nodes': Integer(20, 100, 'uniform', random_state=1),
        'max_depth': Integer(20, 100, 'uniform', random_state=1),
        'max_features': Continuous(0.4, 1.0, 'uniform', random_state=1),
        'max_iter': Integer(100, 300, 'uniform'),
        'l2_regularization': Continuous(0.0, 2.0, 'uniform', random_state=1),
    }

    test_model = HistGradientBoostingClassifier(random_state=1).fit(data, truth)
    grid_search = GASearchCV(test_model, param_grid=search_params_to_use, n_jobs=-1, scoring="f1_macro")
    grid_search.fit(data, truth, callbacks=[pg_bar_callback, delta_callback])
    display(grid_search.best_score_)
    best_params = grid_search.best_params_
    display(grid_search.best_params_)

    test_model = HistGradientBoostingClassifier(random_state=1, **best_params).fit(data.to_numpy(), truth.to_numpy())
    display_cv(test_model, data, truth)
    return best_params

best_hgb_params = {}
if DO_HGB_TUNING:
    best_hgb_params = run_hypertune_test_hgb(cleaned_data, y)
else:
    best_hgb_params = {
        'learning_rate': 0.2331298322064609,
        'class_weight': None,
        'max_leaf_nodes': 26,
        'max_depth': 90,
        'max_features': 0.48875322147097394,
        'max_iter': 119,
        'l2_regularization': 1.6625496693289223
    }

def get_hgb_classifier(data, truth, params_to_use):
    print(f'-- HistGradientBoostingClassifier using: {params_to_use}')
    return HistGradientBoostingClassifier(random_state=1, **params_to_use).fit(data, truth)


### Testing / Tuning with XGBoost

In [ ]:
# do some testing...
DO_RUN_TUNING = False

import xgboost as xgb
from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import RandomizedSearchCV

from sklearn_genetic.space import Continuous, Categorical, Integer
from sklearn_genetic.callbacks import DeltaThreshold

delta_callback = DeltaThreshold(threshold=0.00005, metric='fitness')
def run_hypertune_test(data, truth):

    # tuning process
    # 1. learning rate
    # 2. max_depth and min_child_weight
    # 3. gamma
    # 4. subsample and colsample_bytree
    # 5. then reduce the learning rate
    
    search_params_to_use = {
        'objective': Categorical(['binary:logitraw', 'binary:logistic'], random_state=1),
        'scale_pos_weight': Continuous(2.0, 4.5, 'uniform', random_state=1),
        'min_child_weight': Continuous(0.1, 3.0, 'uniform', random_state=1),
        'learning_rate': Continuous(0.08, 0.3, 'uniform', random_state=1),
        'max_depth': Integer(2, 5, 'uniform', random_state=1),
        'subsample': Continuous(0.9, 1.0, 'uniform', random_state=1),
        'colsample_bytree': Continuous(0.0, 1.0, 'uniform', random_state=1),
        'colsample_bylevel': Continuous(0.0, 1.0, 'uniform', random_state=1),
        'colsample_bynode': Continuous(0.0, 1.0, 'uniform', random_state=1),
        'lambda': Continuous(0.5, 1.5, 'uniform', random_state=1),
        'alpha': Continuous(0.0, 1.5, 'uniform', random_state=1),
        'gamma': Continuous(0.0, 1.5, 'uniform', random_state=1),
        'n_estimators': Integer(50, 500, 'uniform', random_state=1),
    }

    test_model = xgb.XGBClassifier(random_state=1).fit(data.to_numpy(), truth.to_numpy())
    grid_search = GASearchCV(test_model, param_grid=search_params_to_use, n_jobs=-1, scoring="f1_macro")    
    grid_search.fit(data, truth, callbacks=[pg_bar_callback, delta_callback])
    display(grid_search.best_score_)
    best_params = grid_search.best_params_
    display(grid_search.best_params_)

    test_model = xgb.XGBClassifier(random_state=1, **best_params).fit(data.to_numpy(), truth.to_numpy())
    display_cv(test_model, data, truth)
    return best_params

best_xgb_params = {}
if DO_RUN_TUNING:
    best_xgb_params = run_hypertune_test(cleaned_data, y)
else:
    best_xgb_params = {
        'objective': 'binary:logitraw',
        'scale_pos_weight': 3.2737397038037734,
        'min_child_weight': 1.5775380564123773,
        'learning_rate': 0.19208909393473206,
        'max_depth': 3,
        'subsample': 0.9509495881521509,
        'colsample_bytree': 0.5094958815215094,
        'colsample_bylevel': 0.9636708728449709,
        'colsample_bynode': 0.5094958815215094,
        'lambda': 1.0094958815215094,
        'alpha': 0.7642438222822641,
        'gamma': 0.7642438222822641,
        'n_estimators': 419
    }


def get_xgboost_classifier(data, truth, params_to_use = None):
    if params_to_use:
        final_params = params_to_use
    else:
        final_params = {
            # use this to help with the imbalanced targets
            'scale_pos_weight': training_data_pos_scale_weight,
            # it's a binary classifier - this method seems to work best
            'objective': 'binary:logitraw',
        }

    print(f'-- XGBoost using: {final_params}')

    return xgb.XGBClassifier(random_state=1, **final_params).fit(data, truth)

display(best_xgb_params)
test_c = get_xgboost_classifier(cleaned_data, y, best_xgb_params)
display_cv(test_c, cleaned_data, y)


In [ ]:
def run_xgboost(data, truth, params_to_use = None):
    clf = get_xgboost_classifier(data, truth, params_to_use)
    display_cv(clf, data, truth)
    return clf

run_xgboost(cleaned_data, y)
run_xgboost(cleaned_data, y, params_to_use = best_xgb_params )

# note - adding our current FE seems to do worse...


### Multiple classifiers

In [ ]:
from sklearn.ensemble import VotingClassifier, StackingClassifier

def get_voting_classifier(data, truth):
    xgboost_tuned = get_xgboost_classifier(data, truth, best_xgb_params)
    hgb_tuned = get_hgb_classifier(data, truth, best_hgb_params)
    dt_tuned = get_dt_classifier(data, truth, best_dt_params)

    return VotingClassifier([
        # ('xgboost_base', xgboost_base),
        ('xgboost_tuned', xgboost_tuned),
        ('hgb_tuned', hgb_tuned),
        ('dt_tuned', dt_tuned),
    ], voting='soft').fit(data, truth)

def run_voting_classifier(data, truth):
    clf = get_voting_classifier(data, truth)
    display_cv(clf, data, truth)
    return clf

def get_stacking_classifer(data, truth):
    xgboost_tuned = get_xgboost_classifier(data, truth, best_xgb_params)
    hgb_tuned = get_hgb_classifier(data, truth, best_hgb_params)

    return StackingClassifier([
        ('xgboost_tuned', xgboost_tuned),
        ('hgb_tuned', hgb_tuned),
    ], cv=5, n_jobs=-1).fit(data, truth)

def run_stacking_classifier(data, truth):
    clf = get_stacking_classifer(data, truth)
    display_cv(clf, data, truth)
    return clf

clf = run_voting_classifier(cleaned_data, y)
# clf = run_stacking_classifier(cleaned_data, y)


### Let's check our features with our final model - and see what we might want to tune out...

In [ ]:
from sklearn_genetic.genetic_search import GAFeatureSelectionCV

FLAG_FIND_DROP_FEATURES = False

best_features_to_drop = []

def get_tuples_features_to_drop(data, truth):
    final_classifier = get_voting_classifier(data, truth)

    evolved_estimator = GAFeatureSelectionCV(
        estimator=final_classifier,
        cv=5,
        scoring="f1_macro",
        population_size=30,
        generations=20,
        n_jobs=-1,
        verbose=True,
        keep_top_k=2,
        elitism=True,
    )

    input_features = list(data.columns)
    evolved_estimator.fit(data, truth)
    best_features = evolved_estimator.best_features_
    ret = zip(input_features, best_features)
    display(tuple(ret))
    
    from sklearn_genetic.plots import plot_fitness_evolution
    plot_fitness_evolution(evolved_estimator)
    plt.show()

    return ret

def get_list_features_to_drop(data, truth):
    tuples_to_drop = get_tuples_features_to_drop(data, truth)
    features_to_drop = []
    for feature in tuples_to_drop:
        if not feature[1]:
            features_to_drop.append(feature[0])
    display(f'Dropping: {features_to_drop}')
    return features_to_drop

if FLAG_FIND_DROP_FEATURES:
    best_features_to_drop = get_list_features_to_drop(cleaned_data, y)

cleaned_data = cleaned_data.drop(best_features_to_drop, axis=1)

clf = run_voting_classifier(cleaned_data, y)
# clf = run_stacking_classifier(cleaned_data, y)
# clf = run_xgboost(cleaned_data, y, best_xgb_params)


### The above test uses FE on the cleaned data as well - 
Compare the results above with the results before the dropping of features...

## 1.4: Create Predictions for Competition Data

Once we are happy with the estimated performance of our model, we can move on to the final step.

First, we train our model one last time, using all available training data (unlike CV, which always uses a subset). This final training will give our model the best chance as the highest performance.

Then, we must load in the (unlabeled) competition data from the cloud and use our model to generate predictions for each instance in that data. We will then output those predictions to a CSV file. We will then send that file to Steve, and he can then tell us how well we did (because he knows the right answers!).

In [ ]:
# Our model's "final form"

clf = clf.fit(cleaned_data, y)

In [ ]:
X_comp = pd.read_csv("https://drive.google.com/uc?export=download&id=1SmFBoNh7segI1Ky92mfeIe6TpscclMwQ")

# Importantly, we need to perform the same cleaning/transformation steps
# on this competition data as you did the training data. Otherwise, we will
# get an error and/or unexpected results.

X_comp = clean_data(X_comp, use_fe=False)
X_comp = X_comp.drop(best_features_to_drop, axis=1)

# Use your model to make predictions
pred_comp = clf.predict(X_comp)

my_submission = pd.DataFrame({'predicted': pred_comp})

pred_data_pos_scale_weight = get_target_skew_rate(pd.DataFrame(pred_comp, columns=[target_feature]))

display(f'training target skew: {training_data_pos_scale_weight}')
display(f'predicted target skew: {pred_data_pos_scale_weight}')
display(f'difference: {abs(training_data_pos_scale_weight - pred_data_pos_scale_weight)}')

# Let's take a peak at the results (as a sanity check)
display(my_submission.head(10))

# You could use any filename.
my_submission.to_csv('my_submission.csv', index=False)

# You can now download the above file from Colab (see menu on the left)

# Model 2 (Your idea Here!)

Here, you can do all the above, but try different ideas:

- Different ML algorithms (e.g., RandomForestClassifier, LGBM, NN)
- Different data cleaning steps (Ordinal encoding, One Hot Encoding, etc.)
- Hyperparameter tuning (using, e.g., GridSearchCV or RandomizedSearchCV)
- Ensembles
- .... anything you can think of!


Steve's GitHub page is a great place for ideas:

https://github.com/stepthom/869_course

In [ ]:
# TODO: Win the competition here!

# Model 3 (Your next idea here!)

In [ ]:
# TODO: Win the competition here, too!